In [24]:
draft_mode = False
from front_matter import *

First, load some general-purpose Python packages.

In [25]:
import numpy as np # for numerics
import sympy as sp # for symbolics
import control as c # the Control Systems module
import matplotlib as mpl # for plots
import matplotlib.pyplot as plt # also for plots
from IPython.display import display, Markdown, Latex # pretty

The following Python packages are specific for the interactive widget.

In [26]:
from ipywidgets import *
%matplotlib widget

## Symbolic transfer functions

Let's investigate the transfer functions symbolically. We begin by defining the Laplace $s$ and gain symbolic variables.

In [27]:
s,K_p,K_i,K_d = sp.symbols('s K_p K_i K_d')

We will design a PID controller for a plant with the following transfer function.

In [28]:
G_sym = 15000/(s**4+50*s**3+875*s**2+6250*s+15000)
display(G_sym)

15000/(s**4 + 50*s**3 + 875*s**2 + 6250*s + 15000)

The controller has the following symbolic transfer function. 

In [29]:
C_sym = K_p + K_i/s + K_d*s
display(C_sym)

K_d*s + K_i/s + K_p

The closed-loop transfer function for the unity feedback system is as follows.

In [30]:
T_sym = sp.simplify(
  C_sym*G_sym/(1+C_sym*G_sym)
)
T_num, T_den = list( # for simplifying
  map(
    lambda x: sp.collect(x,s),
    sp.fraction(T_sym)
  )
)
T_sym = T_num/T_den
display(T_sym)

(15000*K_i + s*(15000*K_d*s + 15000*K_p))/(15000*K_i + s*(15000*K_d*s + 15000*K_p + s**4 + 50*s**3 + 875*s**2 + 6250*s + 15000))

## Symbolic to `control` transfer functions

The `control` package has objects of type `TransferFunction` that will be useful for simulation in the next section. We begin by defining a function to convert a symbolic transfer function to a `control` `TransferFunction` object.

In [31]:
def sym_to_tf(tf_sym,s_var):
    global s # changes s globally!
    S = s_var
    s = sp.symbols('s')
    tf_sym = tf_sym.subs(S,s)
    tf_str = str(tf_sym)
    s = c.TransferFunction.s
    ldict = {}
    exec('tf_out = '+tf_str,globals(),ldict)
    tf_out = ldict['tf_out']
    return tf_out

This isn't smooth, but it works. Note that `tf_sym` must have no symbolic variables besides `s_var`, the Laplace $s$. We can apply this to `G_sym`, then, but not yet `C_sym`.

In [32]:
type(sym_to_tf(G_sym,s))

control.xferfcn.TransferFunction

## Defining the closed-loop function

We need to create a function that specifies the gains, substitutes them into the symbolic closed-loop transfer function, then converts it to a `control` package `TransferFunction` object via `sym_to_tf`.

In [33]:
def pid_CL_tf(CL_sym,Kp=0,Ki=0,Kd=0):
  sp.symbols('K_p K_i K_d')
  s = c.TransferFunction.s
  CL_subs = CL_sym.subs({K_p: Kp, K_i: Ki, K_d: Kd})
  return sym_to_tf(CL_subs,s)

For instance, we can let $K_p = 1$ and $K_i = K_d = 0$.

In [34]:
display(
  pid_CL_tf(T_sym,Kp=1)
)


                1.5e+04
---------------------------------------
s^4 + 50 s^3 + 875 s^2 + 6250 s + 3e+04

## Step response

It is straightforward to use the `control` package's `step_response` function to get a step response for a single set of gains.

In [35]:
gains = {'Kp':2, 'Ki':1, 'Kd':0.1}
sys_CL = pid_CL_tf(T_sym,**gains)
t_step = np.linspace(0,3,200)
t_step,y_step = c.step_response(sys_CL, t_step)

Now let's plot it. The result is shown in \cref{pid_interactive_design_python_1}.

In [36]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(t_step, y_step)
plt.xlabel('time (s)')
plt.ylabel('step response')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
pxfer(
  fig_name=fig,
  draft_mode=draft_mode,
  filename='pid_interactive_design_python',
  linewidth=1, 
  bbox_inches='tight',
  output_type='pdf',
  figure=True,
  caption=f"step response with $K_p,K_i,K_d = {gains['Kp']},{gains['Ki']},{gains['Kd']}$."
)

<IPython.core.display.Latex object>

## Interactive step response

The following essentially repeats the same process of

1. setting the PID gains with `pid_CL_tf`,
2. simulating with `step_response`, and
3. plotting the response.

The caveat is that this happens with a GUI `interact`ion callback function `update` that sets new gains (based on the GUI sliders), simulates, and replaces the old `line` on the plot.
The final plot is shown in \cref{pid_interactive_design_python_2}. It appears to meet our performance requirements.

In [38]:
%matplotlib widget
# simulate
t_step = np.linspace(0,3,200)
sys_CL = pid_CL_tf(T_sym,Kp=1)
t_step,y_step = c.step_response(sys_CL, t_step)

# initial plot
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(t_step, y_step)
plt.xlabel('time (s)')
plt.ylabel('step response')
plt.show()

# GUI callback function
def update(Kp = 1.0, Ki = 0.0, Kd = 0.0):
    global t_step, kp, ki, kd
    kp,ki,kd = Kp,Ki,Kd
    sys_CL = pid_CL_tf(T_sym,Kp=Kp,Ki=Ki,Kd=Kd)
    t_step,y_step = c.step_response(sys_CL, t_step)
    line.set_ydata(y_step)
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()
    plt.show()

# interaction definition
interact(
  update,
  Kp=(0.0,10.0),
  Ki=(0.0,20.0),
  Kd=(0.0,1.0)
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='Kp', max=10.0), FloatSlider(value=0.0, description='…

In [40]:
pxfer(
  fig_name=fig,
  draft_mode=draft_mode,
  filename='pid_interactive_design_python',
  linewidth=1, 
  bbox_inches='tight',
  output_type='pdf',
  figure=True,
  caption="step response from interaction" \
    " with $K_p,K_i,K_d = " \
    f"{kp},{ki},{kd}$.",
  number=2
)

<IPython.core.display.Latex object>

The sliders appear as shown below.

```{=latex}
\begin{center}
\includegraphics[width=2in]{source/pid_interactive_design_python_3.jpg}
\end{center}
```